In [14]:
import sys
import os
from typing import Dict, Tuple
from src.utils import default_data_path, config_to_env, check_config
from src.task import LaMPTask
from src.models import (
    feed_prompt_to_lm,
    feed_prompts_to_lm,
    OpenAIModel,
    task_1_parse_response,
    DistilBERTModel,
    BERTSERINIModel,
    MiniLM,
)

In [15]:
config_to_env("OPENAI_API_KEY")

In [16]:
task_header = "LaMP_1"
store_folder = os.path.join("src", "data", task_header)

OpenAI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=OpenAIModel(),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "OpenAI.txt"),
    callback=task_1_parse_response,
)
DistilBERT_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=DistilBERTModel(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "DistilBERTModel.txt"),
    callback=task_1_parse_response,
)
BERTSERINI_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=BERTSERINIModel(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "BERTSERINI.txt"),
    callback=task_1_parse_response,
)
MiniLM_Subscriber = lambda id, prompt, api_key: feed_prompt_to_lm(
    model=MiniLM(task_name="LaMP_1"),
    id=id,
    prompt=prompt,
    api_key=api_key,
    log_path=os.path.join(store_folder, "MiniLM.txt"),
    callback=task_1_parse_response,
)

In [29]:
def task_1_with_keywords_query(with_keyword_params, debug: bool = False):
    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    os.makedirs(store_dir, exist_ok=True)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for text_rank_top_k_keywords, bm25_top_k in with_keyword_params:
        file_ending = f"{entry_per_category}_{text_rank_top_k_keywords}_{bm25_top_k}"

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_with_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                # "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                "OpenAI": OpenAI_Subscriber,
            },
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path={
                # "BERTSERINI": os.path.join(store_dir,f"{task_header}_train_preds_BERTSERINI_with_keyword_{file_ending}.json"),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_with_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_with_keyword_{file_ending}.json",
                # ),
                "OpenAI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_OpenAI_with_keyword_{file_ending}.json",
                ),
            },
            keyword_extraction=True,
        )

        ready_model[(text_rank_top_k_keywords, bm25_top_k)] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_1_with_keywords_eval(
    with_keyword_params, ready_models: Dict[Tuple[int, int], LaMPTask] = None
):
    if ready_models is not None:
        for (text_rank_top_k_keywords, bm25_top_k), curr_task in ready_models.items():
            curr_task.evaluate()
        return

    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120

    evaluated_result = dict()

    for text_rank_top_k_keywords, bm25_top_k in with_keyword_params:
        file_ending = f"{entry_per_category}_{text_rank_top_k_keywords}_{bm25_top_k}"
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_with_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                "BERTSERINI": BERTSERINI_Subscriber,
                "DistilBERT": DistilBERT_Subscriber,
                "MiniLM": MiniLM_Subscriber,
                "OpenAI": OpenAI_Subscriber,
            },
        )
        curr_task.evaluate(
            preds_save_name={
                "BERTSERINI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_BERTSERINI_with_keyword_{file_ending}.json",
                ),
                "DistilBERT": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_DistilBERT_with_keyword_{file_ending}.json",
                ),
                "MiniLM": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_MiniLM_with_keyword_{file_ending}.json",
                ),
                "OpenAI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_OpenAI_with_keyword_{file_ending}.json",
                ),
            }
        )
        # print(curr_task.score)
        evaluated_result[f"{text_rank_top_k_keywords} {bm25_top_k}"] = curr_task.score
    return evaluated_result

In [28]:
with_keyword_params = [(5, 5), (10, 10)]
debug = False

In [30]:
# with_keyword_params = [(5, 5)]
# with_keyword_params = [(10, 10)]
# debug=True

In [31]:
# ready_models = task_1_with_keywords_query(with_keyword_params, debug=debug)

In [32]:
evaluate_results = task_1_with_keywords_eval(with_keyword_params)

In [38]:
def task_1_without_keywords_query(without_keyword_params, debug: bool = False):
    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    os.makedirs(store_dir, exist_ok=True)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120
    worker_count = 8 if not debug else 1
    api_keys = (
        [
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
            check_config("HUGGING_FACE_KEY_1"),
            check_config("HUGGING_FACE_KEY_2"),
            check_config("HUGGING_FACE_KEY_3"),
            check_config("HUGGING_FACE_KEY_4"),
        ]
        if not debug
        else None
    )

    ready_model = dict()

    for bm25_top_k in without_keyword_params:
        file_ending = f"{entry_per_category}_{bm25_top_k}"

        question_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_prompts_questions_without_keyword_{file_ending}.json",
        )
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                "BERTSERINI": BERTSERINI_Subscriber,
                # "DistilBERT": DistilBERT_Subscriber,
                # "MiniLM": MiniLM_Subscriber,
                # "OpenAI": OpenAI_Subscriber,
            },
            worker_count=worker_count,
            prompt_save_path=question_store_path,
            preds_save_path={
                "BERTSERINI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_BERTSERINI_without_keyword_{file_ending}.json",
                ),
                # "DistilBERT": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_DistilBERT_without_keyword_{file_ending}.json",
                # ),
                # "MiniLM": os.path.join(
                #     store_dir,
                #     f"{task_header}_train_preds_MiniLM_without_keyword_{file_ending}.json",
                # ),
                # "OpenAI": os.path.join(store_dir,f"{task_header}_train_preds_OpenAI_without_keyword_{file_ending}.json"),
            },
            keyword_extraction=False,
        )

        ready_model[bm25_top_k] = curr_task
        curr_task.load_prompts(question_store_path)
        curr_task.subscribe(
            skip_eval=True,
            api_keys=api_keys,
        )


def task_1_without_keywords_eval(
    without_keyword_params, ready_models: Dict[Tuple[int, int], LaMPTask] = None
):
    if ready_models is not None:
        for bm25_top_k, curr_task in ready_models.items():
            curr_task.evaluate()
        return

    task_header = "LaMP_1"
    store_dir = os.path.join("src", "data", task_header)
    dataset_question_path = os.path.join("src", "data", "LaMP_1_train_questions.json")
    entry_per_category = 120

    evaluated_result = dict()

    for bm25_top_k in without_keyword_params:
        file_ending = f"{entry_per_category}_{bm25_top_k}"
        output_store_path = os.path.join(
            store_dir,
            f"{task_header}_train_outputs_selected_without_keyword_{file_ending}.json",
        )
        curr_task = LaMPTask(
            task_question_file=dataset_question_path,
            task_output_file=output_store_path,
            subscribers={
                "BERTSERINI": BERTSERINI_Subscriber,
                "DistilBERT": DistilBERT_Subscriber,
                "MiniLM": MiniLM_Subscriber,
                "OpenAI": OpenAI_Subscriber,
            },
        )
        curr_task.evaluate(
            preds_save_name={
                "BERTSERINI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_BERTSERINI_without_keyword_{file_ending}.json",
                ),
                "DistilBERT": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_DistilBERT_without_keyword_{file_ending}.json",
                ),
                "MiniLM": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_MiniLM_without_keyword_{file_ending}.json",
                ),
                "OpenAI": os.path.join(
                    store_dir,
                    f"{task_header}_train_preds_OpenAI_without_keyword_{file_ending}.json",
                ),
            }
        )
        print(curr_task.score)
        evaluated_result[f"{bm25_top_k}"] = curr_task.score
    return evaluated_result

In [39]:
debug = False
without_keyword_params = [2, 4]
# without_keyword_params = [4]

In [36]:
# ready_models_no_keywords = task_1_without_keywords_query(
#     without_keyword_params, debug=debug
# )

In [40]:
evaluate_results_no_keywords = task_1_without_keywords_eval(without_keyword_params)

{'BERTSERINI': {'accuracy': 0.41201716738197425, 'f1': 0.3752169815999603}, 'DistilBERT': {'accuracy': 0.40772532188841204, 'f1': 0.37799707602339183}, 'MiniLM': {'accuracy': 0.6266094420600858, 'f1': 0.6276977751453003}, 'OpenAI': {'accuracy': 0.22317596566523606, 'f1': 0.2934027777777778}}
{'BERTSERINI': {'accuracy': 0.3191489361702128, 'f1': 0.2987044882421563}, 'DistilBERT': {'accuracy': 0.35319148936170214, 'f1': 0.3541926110419261}, 'MiniLM': {'accuracy': 0.6212765957446809, 'f1': 0.6252705329958549}, 'OpenAI': {'accuracy': 0.1829787234042553, 'f1': 0.27045424211416114}}


In [42]:
import json

print(
    json.dumps(
        evaluate_results,
        indent=4,
    )
)
print(
    json.dumps(
        evaluate_results_no_keywords,
        indent=4,
    )
)

{
    "5 5": {
        "BERTSERINI": {
            "accuracy": 0.5042016806722689,
            "f1": 0.34267302426588614
        },
        "DistilBERT": {
            "accuracy": 0.5042016806722689,
            "f1": 0.35189393939393937
        },
        "MiniLM": {
            "accuracy": 0.5294117647058824,
            "f1": 0.5122202591283863
        },
        "OpenAI": {
            "accuracy": 0.5336134453781513,
            "f1": 0.5394104308390022
        }
    },
    "10 10": {
        "BERTSERINI": {
            "accuracy": 0.4641350210970464,
            "f1": 0.3702348690153568
        },
        "DistilBERT": {
            "accuracy": 0.4177215189873418,
            "f1": 0.3884012539184953
        },
        "MiniLM": {
            "accuracy": 0.6033755274261603,
            "f1": 0.5996046010064702
        },
        "OpenAI": {
            "accuracy": 0.5063291139240507,
            "f1": 0.5218125960061444
        }
    }
}
{
    "2": {
        "BERTSERINI": {
      